In [ ]:
import gurobipy as grb
from gurobipy import *
import numpy as np

In [ ]:
class SPP():
    def __init__(self, num_intervals, num_pairs, num_links, impact_intervals):
        self.num_intervals = num_intervals
        self.num_pairs = num_pairs
        self.num_links = num_links
        self.imp_intervals = impact_intervals
    
    
    def build_model(self, prior_od, prior_link, assignment_fraction, mcr):
        
        assert prior_od.shape == (self.num_intervals, self.num_pairs)
        assert prior_link.shape == (self.num_intervals, self.num_links)
        assert assignment_fraction.shape == (self.num_pairs, self.num_links, self.imp_intervals)
        
        # create model
        spp = Model("SPP")
        
        # create varibles
        X = spp.addVars(['X_' + str(i) for i in range(self.num_intervals * self.num_pairs)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        Y = spp.addVars(['Y_' + str(i) for i in range(self.num_intervals * self.num_links)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        X = np.array(X.values()).reshape(self.num_intervals, self.num_pairs)
        Y = np.array(Y.values()).reshape(self.num_intervals, self.num_links)
        spp.update()
        
        # build objective
        X_hat = prior_od
        Y_hat = prior_link
        X_var = np.var(prior_od)
        Y_var = np.var(prior_link)
        
        obj = 0
        for k in range(self.num_intervals):
            for i in range(self.num_pairs):
                obj += (X[k,i] - X_hat[k,i]) * (X[k,i] - X_hat[k,i]) / X_var
        
        for k in range(self.num_intervals):
            for j in range(self.num_links):
                obj += (Y[k,j] - Y_hat[k,j]) * (Y[k,j] - Y_hat[k,j]) / Y_var
        
        spp.setObjective(obj, GRB.MINIMIZE)
        
        # add constraints
        # No.1: Linear Assignment
        T = self.imp_intervals
        A = assignment_fraction
        for k in range(self.num_intervals):
            t = 0
            rhs = 0
            while (k - t >= 0) & (t <= T):
                rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,self.num_links)
                t += 1
            for j in range(self.num_links):
                spp.addConstr(Y[k,j] == rhs[0,j])
        
        # No.2: Maximum Change Ratio
        beta = mcr
        for k in range(1,self.num_intervals):
            for i in range(self.num_pairs):
                spp.addConstr(X[k,i] >= (1 - beta) * X[k - 1,i])
                spp.addConstr(X[k,i] <= (1 + beta) * X[k - 1,i])
        
        return spp

    
    def estimate(self, prior_od, prior_link, assignment_fraction, mcr):
        
        spp = self.build_model(prior_od, prior_link, assignment_fraction, mcr)
        spp.optimize()
        
        x_range = self.num_intervals * self.num_pairs
        solution_x = [v.x for v in spp.getVars()[:x_range]]
        solution_y = [v.x for v in spp.getVars()[x_range:]]
        
        return solution_x, solution_y

In [ ]:
prior_od = 5 + 5 * np.random.rand(num_intervals, num_pairs)
prior_link = 100 + 10 * np.random.rand(num_intervals, num_links)
prior_pr = np.random.rand(num_intervals, num_pairs)
assignment_fraction = np.random.rand(num_pairs,num_links,impact_intervals)

In [ ]:
# Create a new model
PRA = Model("PRA")

# Create variables
num_intervals = 18
num_pairs = 120
num_links = 4
impact_intervals = 3 # 1 + (max travel time // interval length)

x = PRA.addVars(['x_' + str(i) for i in range(num_pairs)], lb=0, obj=1, vtype=GRB.CONTINUOUS)
y = PRA.addVars(['y_' + str(i) for i in range(num_links)],lb=0, obj=1, vtype=GRB.CONTINUOUS)
z = PRA.addVars(['z_' + str(i) for i in range(num_pairs)],lb=0, obj=1, vtype=GRB.CONTINUOUS)
t = PRA.addVars(['t_' + str(i) for i in range(num_pairs)],lb=0, ub=1, obj=1, vtype=GRB.CONTINUOUS)

# Integrate new variables
PRA.update()

In [ ]:
obj_term_od = x.values() - prior_od
obj_term_od *= obj_term_od
obj_term_link = y.values() - prior_link
obj_term_link *= obj_term_link
obj_term_pr = z.values() - prior_pr
obj_term_pr *= obj_term_pr


obj_term = 0
for term in obj_term_od:
    obj_term += term
for term in obj_term_link:
    obj_term += term
for term in obj_term_pr:
    obj_term += term

# Set objective
PRA.setObjective(obj_term, GRB.MINIMIZE)

In [ ]:
# Add constraints
PRA.params.NonConvex = 2


# Constraint #1: traffic assignment
incidence_matrix = np.round(np.random.rand(len(x),len(y)))
assignment_fraction = np.random.rand(len(x),len(y))
for idx, yy in enumerate(y.values()):
    rhs = x.values() * incidence_matrix[:,idx] * assignment_fraction[:,idx]
    rhs_sum = 0
    for term in rhs:
        rhs_sum += term
    PRA.addConstr(yy == rhs_sum)


# Constraint #2: Penetration rate assignment
for idx, xx in enumerate(x.values()):
    upside = np.random.rand()
    PRA.addConstr(t.values()[idx] * xx - upside <= 1)
    PRA.addConstr(t.values()[idx] * xx - upside >= -1)

incidence_matrix_pr = np.round(np.random.rand(len(t),len(z)))
assignment_fraction_pr = np.random.rand(len(t),len(z))
for idx, zz in enumerate(z.values()):
    rhs = t.values() * incidence_matrix_pr[:,idx] * assignment_fraction_pr[:,idx]
    rhs_sum = 0
    for term in rhs:
        rhs_sum += term
    PRA.addConstr(zz - rhs_sum <= 0.5)
    PRA.addConstr(zz - rhs_sum >= -0.5)

In [ ]:
PRA.optimize()

In [ ]:
solution_x = [v.x for v in PRA.getVars()[:num_pairs]]
solution_y = [v.x for v in PRA.getVars()[num_pairs:(num_links + num_pairs)]]
solution_z = [v.x for v in PRA.getVars()[(num_links + num_pairs):(num_links + 2 * num_pairs)]]